In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import re

In [2]:
df_with_id = pd.read_csv('data/population_cleaned.csv')


In [3]:
df = df_with_id.drop('id', axis=1)

In [4]:
df.head()

,age,nationality,sex,commune,year,value
0,0,S,M,Aeugst am Albis,2010,8
1,1,S,M,Aeugst am Albis,2010,9
2,2,S,M,Aeugst am Albis,2010,7
3,3,S,M,Aeugst am Albis,2010,13
4,4,S,M,Aeugst am Albis,2010,5


In [5]:
#group the ages by categories :{[0, 18), [18, 40), [40, 65), [65, 100)}
d = pd.DataFrame(columns=[ 'commune', 'sex', 'year', 'nationality', 'age', 'value'])
for l, u in [(0, 18), (18, 40), (40, 65), (65, 100)]:
    tmp = df[df['age'] >= l]
    tmp = tmp[tmp['age'] < u]
    tmp = tmp.groupby([ 'commune', 'sex', 'year', 'nationality'], as_index=False).sum()
    tmp['age'] = int(u)
    d = pd.concat([d, tmp])
for s in ['year', 'age', 'value']:
    d[s] = d[s].astype(int)
df = d

In [6]:
df.head()

,commune,sex,year,nationality,age,value
0,Aadorf,F,2010,A,18,122
1,Aadorf,F,2010,S,18,650
2,Aadorf,F,2011,A,18,120
3,Aadorf,F,2011,S,18,656
4,Aadorf,F,2012,A,18,108


In [7]:
ids6 =  pd.read_csv('data/data_to_be_processed/ids')
ids6.rename(columns={'name':'commune'}, inplace=True)
ids = df_with_id.drop(['sex', 'year', 'nationality', 'age', 'value'], axis=1)
ids = pd.concat([ids, ids6])
ids = ids.drop_duplicates()

no data are available for 2016. We have therefore decided to keep the data from 2015 and sum the data of the municipalities that have merged in beginning of 2016

In [8]:
def create_2016_data(data):
    f = pd.read_csv("data/fusion_mappings.csv")
    f = f[f.date.str.contains('2016')]
    fu = pd.merge(f, data, left_on='pre_name', right_on='commune', how='outer')
    fu['commune'] = fu['post_name'].fillna(fu['commune'])
    fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
    fu = fu.groupby(['commune', 'sex', 'nationality', 'age'], as_index=False).sum()
    return fu

In [9]:
for year in range (2013, 2017):    
    #filter out by year
    if(year == 2016):
        tmp = df[df['year'] == 2015]
        tmp = tmp.drop('year', axis=1)
        tmp = create_2016_data(tmp)
    else:
        tmp = df[df['year'] == year]
        tmp = tmp.drop('year', axis=1)
        
    #compute total population by municipality
    total = tmp.groupby('commune').sum()
    total = total.drop('age', axis=1)
    total.columns = ['total_inhabitants']
    
    #for each age category compute the percentage of inhabitants in it and add the row to total
    for i in [18, 40, 65, 100]:
        age = tmp[tmp['age'] == i].groupby('commune').sum()
        age = age.drop('age', axis=1)
        age.columns = ['percentage_%d' %i]
        total = pd.merge(total, age, left_index=True, right_index=True)
        total['percentage_%d' %i] = total['percentage_%d' %i] / total['total_inhabitants']
    
    #compute percentage of men in each municipality
    men = tmp[tmp.sex == 'M'].groupby('commune').sum()
    men = men.drop('age', axis=1)
    men.columns = ['percentage_men']
    total = pd.merge(total, men, left_index=True, right_index=True)
    total['percentage_men'] =  total['percentage_men'] / total['total_inhabitants']
    
    #compute percentage of swissin each municipality
    swiss = tmp[tmp.nationality == 'S'].groupby('commune').sum()
    swiss = swiss.drop('age', axis=1)
    swiss.columns = ['percentage_swiss']
    total = pd.merge(total, swiss, left_index=True, right_index=True)
    total['percentage_swiss'] = total['percentage_swiss'] / total['total_inhabitants']
    total = total.reset_index(level=0)
    
    #merge dataframe with municiplity ids
    total = pd.merge(ids, total, left_on='commune', right_on='commune', how='right')
    
    #check that we have no empty values
    inds = total[pd.isnull(total).any(1)]
    assert(inds.empty)
    
    #round percentages to 2 decimals
    total = total.round(2)
    total.to_csv('data/municipalities/%d/data_commune.csv' % year, index=False)

In [10]:
#data for 2016
total.head()

,commune,id,total_inhabitants,percentage_18,percentage_40,percentage_65,percentage_100,percentage_men,percentage_swiss
0,Aeugst am Albis,1,1981,0.19,0.23,0.42,0.16,0.50,0.87
1,Affoltern am Albis,2,11707,0.18,0.30,0.35,0.17,0.50,0.73
2,Bonstetten,3,5326,0.22,0.26,0.37,0.15,0.49,0.86
3,Hausen am Albis,4,3477,0.20,0.23,0.40,0.17,0.49,0.86
4,Hedingen,5,3659,0.21,0.25,0.39,0.15,0.50,0.85
